In [7]:
%matplotlib inline
import pandas as pd
import os
os.chdir('/Users/zhouyu/Google Drive/python4work/')

In [10]:
dataset = pd.read_csv("exc_inh_spikes19-Dec-2016.csv")
dataset['Loc_x'] = dataset.apply(lambda row:int(row['splitOnPatchLocation'].split('  ')[0]),axis = 1)
dataset['Loc_y'] = dataset.apply(lambda row:int(row['splitOnPatchLocation'].split('  ')[1]),axis = 1)
dataset['cell_id'] = dataset['splitOnExperimentDate']+dataset['celllabel']
dataset = dataset.drop(['splitOnPatchLocation','splitOnDelTag','nodeid','leafid','splitOnExperimentDate','celllabel'],axis = 1)
dataset = dataset.drop(['currentPatchLocation_1','currentPatchLocation_2'],axis = 1)
dataset.head(3)

,id,splitOnRecType,stimulusTag,imageName,imagePatchIndex,equivalentIntensity,backgroundIntensity,MeanRespOn,SEM,Loc_x,Loc_y,cell_id
0,1,exc,image,152,1,0.328417,0.199407,-822.143387,43.759201,1241,648,2016/08/10Cell 1- on parasol
1,2,exc,intensity,152,1,0.328417,0.199407,-673.196247,106.060358,1241,648,2016/08/10Cell 1- on parasol
2,3,exc,image,152,2,0.200514,0.199407,-57.449961,20.437119,679,707,2016/08/10Cell 1- on parasol


In [11]:
import numpy as np
dataset = dataset[dataset['imageName']==152]
dataset = dataset[dataset['splitOnRecType']!='cell-attach']
dataset = dataset.drop(['id'],axis = 1)
dataset.shape

(1728, 11)

In [12]:
exc = dataset[dataset['splitOnRecType']=='exc']
inh = dataset[dataset['splitOnRecType']=='inh']
ei_table = pd.merge(exc,inh,how = 'inner',on = ['cell_id','imageName','Loc_x','Loc_y','stimulusTag','equivalentIntensity','backgroundIntensity'])
ei_table['IE_ratio'] = ei_table.apply(lambda row:(-row['MeanRespOn_y']/row['MeanRespOn_x']),axis = 1)
ei_table.head(3)

,splitOnRecType_x,stimulusTag,imageName,imagePatchIndex_x,equivalentIntensity,backgroundIntensity,MeanRespOn_x,SEM_x,Loc_x,Loc_y,cell_id,splitOnRecType_y,imagePatchIndex_y,MeanRespOn_y,SEM_y,IE_ratio
0,exc,image,152,1,0.326865,0.199407,-568.351463,40.194951,1241,648,2016/08/10Cell 2 - on parasol,inh,1,662.307364,134.002112,1.165313
1,exc,intensity,152,1,0.326865,0.199407,-560.141950,41.428937,1241,648,2016/08/10Cell 2 - on parasol,inh,1,613.873961,191.138438,1.095926
2,exc,image,152,2,0.201769,0.199407,-52.767562,17.030589,679,707,2016/08/10Cell 2 - on parasol,inh,2,1864.584197,395.481852,35.335803


In [13]:
ei_table.drop(['splitOnRecType_x','splitOnRecType_y'],axis = 1)
ei_table.rename(columns = {'MeanRespOn_x':'Exc_Resp','SEM_x':'Exc_SEM','MeanRespOn_y':'Inh_Resp','SEM_y':'Inh_SEM'}, inplace = True)
ei_table.to_csv('IE_ratio1219.csv')

In [18]:
contrast=pd.read_csv("exc_inh_contrast19-Dec-2016.csv")
contrast['cell_id'] = contrast['splitOnExperimentDate']+contrast['celllabel']
contrast = contrast.drop(['splitOnExperimentDate','celllabel','id','backgroundIntensity','nodeid','leafid','splitOnDelTag'],axis = 1)
exc = contrast[contrast['splitOnRecType']=='exc']
inh = contrast[contrast['splitOnRecType']=='inh']
contrast_table = pd.merge(exc,inh,how = 'inner',on = ['cell_id','currentSpotContrast',])

In [22]:
contrast_table.head(3)

,splitOnRecType_x,currentSpotContrast,MeanExc,SEM_Exc,cell_id,splitOnRecType_y,MeanInh,SEM_Inh
0,exc,-0.90,37.568833,6.892708,2016/09/13On parasol1,inh,1351.621646,86.832268
1,exc,-0.75,35.964670,4.584801,2016/09/13On parasol1,inh,1256.355670,88.001208
2,exc,-0.50,44.267558,4.891360,2016/09/13On parasol1,inh,1027.190100,64.992141


In [20]:
contrast_table.rename(columns = {'MeanRespOn_x':'MeanExc','SEM_x':'SEM_Exc','MeanRespOn_y':'MeanInh','SEM_y':'SEM_Inh'},inplace = True)

In [24]:
contrast_table.to_csv("contrast_inh_exc.csv")